In [0]:
spark.conf.set("spark.hadoop.fs.azure.account.key.projecthockey.blob.core.windows.net", "6+l6fbr+fm+FI4xRaH8KxnRcju4kyXcQsyOl7UJmy1y/P3VY/2M7j2LihOPcRg1w9CMK8uTrIr5g+AStsXRXsA==")


In [0]:
dbutils.fs.unmount("/mnt/hockey1")

In [0]:
configs = {
    "fs.azure.account.key.projecthockey.blob.core.windows.net": "6+l6fbr+fm+FI4xRaH8KxnRcju4kyXcQsyOl7UJmy1y/P3VY/2M7j2LihOPcRg1w9CMK8uTrIr5g+AStsXRXsA=="
}

# all above configurations are used to create mount point
dbutils.fs.mount(
  source = "wasbs://hockey@projecthockey.blob.core.windows.net/",
  mount_point = "/mnt/hockey1",
  extra_configs = configs)

True

In [0]:
from pyspark.sql.functions import col

df_players = spark.read.csv("/mnt/hockey1/Hockey_Src1.csv", header=True, inferSchema=True)
adrian_info = df_players.filter((col("Player Name") == "Adrian") & (col("Club Name")=="Real Sociedad") )




In [0]:
df_players.count()


10800

In [0]:
df_matches = spark.read.csv("/mnt/hockey1/Hockey_Src2.csv", header=True, inferSchema=True)
df_matches.count()


1440

In [0]:
df_players.show()

+----+---------+-------------+-----------+---------+------------+--------+---------+---------+-----------+-----------+------------+------------+-----------+---------------+----------+--------------+-----------+--------+-----------+-----------------------+-------------+------------------+---------+
|Year|    Month|        Match|Player Name|Club Name|country Name|Position|Jersey_No| D. O. B.|Nationality|appearances|goals scored|goals assist|total shots|shots on target|fouls made|fouls suffered|yellow card|red card|goals saved|goals conceded(stopped)|total penalty|successful penalty|   salary|
+----+---------+-------------+-----------+---------+------------+--------+---------+---------+-----------+-----------+------------+------------+-----------+---------------+----------+--------------+-----------+--------+-----------+-----------------------+-------------+------------------+---------+
|2010|  January|International|     Adrian|       NA| Netherlands|      LW|       88|30-Mar-85|Netherlan

In [0]:
df_date = spark.read.csv("/mnt/hockey1/DIM.Date.Table.csv", header=True, inferSchema=True)
df_date.show()

+--------+-------------------+-----------+----------------+---------------+---------+----------+------------+----------------+----------------+-------------------+-------------------+--------+-----------------+----------+------------+-------+-------+------+------------+--------------+-----------+----------------------+----------------------+
|date_key|          full_date|day_of_week|day_num_in_month|day_num_overall| day_name|day_abbrev|weekday_flag|week_num_in_year|week_num_overall|    week_begin_date|week_begin_date_key|fb_month|month_num_overall|month_name|month_abbrev|quarter|fb_year|yearmo|fiscal_month|fiscal_quarter|fiscal_year|last_day_in_month_flag|same_day_year_ago_date|
+--------+-------------------+-----------+----------------+---------------+---------+----------+------------+----------------+----------------+-------------------+-------------------+--------+-----------------+----------+------------+-------+-------+------+------------+--------------+-----------+---------------

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
# Load Source File 2
source_file_2 = spark.read.csv("/mnt/hockey1/Hockey_Src2.csv", header=True, inferSchema=True)


spark.conf.set(
    "fs.azure.account.key.projecthockey.blob.core.windows.net",
    "6+l6fbr+fm+FI4xRaH8KxnRcju4kyXcQsyOl7UJmy1y/P3VY/2M7j2LihOPcRg1w9CMK8uTrIr5g+AStsXRXsA=="
)

# Load Source File 2
source_file_2 = spark.read.csv(
    "/mnt/hockey1/Hockey_Src2.csv",
    header=True,
    inferSchema=True
)

# Extract League Data
league_dim = source_file_2.select("League_Name").distinct() \
    .withColumnRenamed("League Name", "League_Name") \
    .withColumn("League_ID", monotonically_increasing_id())

# Verify the League Dimension
league_dim.show()
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/Dimensions/league_dim/"

# Write DataFrame to the container
league_dim.write.format("parquet").mode("overwrite").save(output_path)



+-----------+---------+
|League_Name|League_ID|
+-----------+---------+
|         NA|        0|
| Bundesliga|        1|
|        EPL|        2|
|    LA Liga|        3|
+-----------+---------+



In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Load both source files
source_file_1 = spark.read.csv("/mnt/hockey1/Hockey_Src1.csv", header=True, inferSchema=True)
source_file_2 = spark.read.csv("/mnt/hockey1/Hockey_Src2.csv", header=True, inferSchema=True)
source_file_1 = source_file_1.withColumnRenamed("Club Name", "Club_Name")
source_file_2 = source_file_2.withColumnRenamed("Club_Name", "Club_Name")
# Extract and clean up the Club data from both files
club_data_1 = source_file_1.select("Club_Name").distinct()  # From source file 1
club_data_2 = source_file_2.selectExpr("Club_Name", "Manager", "Owner").distinct()
# Join the two datasets on Club_Name to combine all attributes
club_data_combined = club_data_1.join(club_data_2, "Club_Name", "left")  # Using left join to keep all clubs
club_data_combined=club_data_combined.dropDuplicates(["Club_Name","Manager","Owner"])
# Generate unique Club_ID
club_data_with_ids = club_data_combined.withColumn("Club_ID", monotonically_increasing_id())
club_data_combined=club_data_combined.dropDuplicates(["Club_Name","Manager","Owner"])
# Optionally, if you want to add League_ID, you can either manually assign it or generate it in a similar way
# For now, we'll assume the League_ID can be generated in the same way (or populated separately if needed)
club_data_with_ids = club_data_with_ids.withColumn("League_ID", monotonically_increasing_id())

# Show the resulting data for verification
club_data_with_ids.show()

# Define the output path for the Club Dimension in Azure Blob Storage
output_path_club = f"wasbs://hockey@projecthockey.blob.core.windows.net/Dimensions/club_dim/"

# Write the Club Dimension DataFrame to the container in Parquet format
club_data_with_ids.write.format("parquet").mode("overwrite").save(output_path_club)

+-----------------+----------------+------------------+-------+---------+
|        Club_Name|         Manager|             Owner|Club_ID|League_ID|
+-----------------+----------------+------------------+-------+---------+
|     Swanswa City|  Duncan Navarro|        Caleb Cote|      0|        0|
|       Sunderland| Zachary Burgess|  Marsden Clements|      1|        1|
|          FC Koln|  Dante Castillo|      Cyrus Sutton|      2|        2|
|           ManUtd|Theodore Mcguire|   Baxter Stephens|      3|        3|
|          Schalke|     Wanda Mayer|   Adam Strickland|      4|        4|
|       Sevilla FC|       Tad Glenn|     Hamilton Cobb|      5|        5|
|        Liverpool|     Kato Carver|      Wyatt Joyner|      6|        6|
|         Valencia|  Phelan Rollins|       Fuller Odom|      7|        7|
|     Hamburger SV| Chelsea Mendoza|     Anthony Weeks|      8|        8|
|    Real Sociedad|    Edward Chase|   Wyatt Hernandez|      9|        9|
|      Real Madrid|     Tad Gardner|Ch

In [0]:

source_file_1 = source_file_1.withColumnRenamed("country Name", "country_Name")
source_file_2 = source_file_2.withColumnRenamed("Country Name", "country_Name")
country_data_1 = source_file_1.select("country_Name").distinct()

# File 2 contains Country_Name and Coach
country_data_2 = source_file_2.select("country_Name", "Coach").distinct()

# Step 3: Join the two datasets on Country_Name to get all columns in one DataFrame
country_data_combined = country_data_1.join(country_data_2, "country_Name", "left")
country_data_combined=country_data_combined.dropDuplicates(["country_Name","Coach"])
# Step 4: Generate unique Country_ID using monotonically_increasing_id
country_data_with_ids = country_data_combined.withColumn("Country_ID", monotonically_increasing_id())

# Step 5: Verify the Country Dimension
country_data_with_ids.show()

# Step 6: Define the output path for saving the Country Dimension as CSV
output_path_country_csv = f"wasbs://hockey@projecthockey.blob.core.windows.net/Dimensions/country_dim_csv/"

# Step 7: Write the DataFrame to CSV in Azure Blob Storage
country_data_with_ids.write.format("csv").option("header", "true").mode("overwrite").save(output_path_country_csv)

+------------+----------------+----------+
|country_Name|           Coach|Country_ID|
+------------+----------------+----------+
|      Brasil|   Cody Delacruz|         0|
|     England| Tobias Melendez|         1|
|     Belgium|    Lucas Hodges|         2|
|       Spain|Prescott Sweeney|         3|
|    Portugal|   William Price|         4|
|   Argentina|    Damon Pruitt|         5|
|     Germany|  Barry Reynolds|         6|
|          NA|              NA|         7|
|      Mexico|  Kuame Gallegos|         8|
|       Italy|    Elmo Patrick|         9|
| Netherlands|      Will Smith|        10|
+------------+----------------+----------+



In [0]:
match_data_1 = source_file_1.select("Match").distinct() \
    .withColumnRenamed("Match", "Match_Name")

match_data_2 = source_file_2.select("Match_Name").distinct()

# Combine the match data from both files using a full outer join
match_data_combined = match_data_1.join(match_data_2,"Match_Name", "left")
match_data_combined=match_data_combined.dropDuplicates(["Match_Name"])
# Generate unique Match IDs using monotonically_increasing_id
match_dim = match_data_combined.withColumn("Match_ID", monotonically_increasing_id())

# Show the resulting match_dim
match_dim.show()

# Write the result to a container
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/Dimensions/match_dim/"
match_dim.write.format("parquet").mode("overwrite").save(output_path)

+-------------+--------+
|   Match_Name|Match_ID|
+-------------+--------+
|International|       0|
|         Club|       1|
+-------------+--------+



In [0]:
source_file_1.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- Match: string (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Club_Name: string (nullable = true)
 |-- country_Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey_No: integer (nullable = true)
 |-- D. O. B.: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- appearances: integer (nullable = true)
 |-- goals scored: integer (nullable = true)
 |-- goals assist: integer (nullable = true)
 |-- total shots: integer (nullable = true)
 |-- shots on target: integer (nullable = true)
 |-- fouls made: integer (nullable = true)
 |-- fouls suffered: integer (nullable = true)
 |-- yellow card: integer (nullable = true)
 |-- red card: integer (nullable = true)
 |-- goals saved: integer (nullable = true)
 |-- goals conceded(stopped): integer (nullable = true)
 |-- total penalty: integer (nullable = true)
 |-- successful penalty: integer (nullable =

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, expr

source_file_1 = spark.read.csv("/mnt/hockey1/Hockey_Src1.csv", header=True, inferSchema=True)
source_file_1 = source_file_1.withColumnRenamed("D. O. B.", "DOB")
# Load Club Dimension
club_dim = spark.read.parquet("wasbs://hockey@projecthockey.blob.core.windows.net/Dimensions/club_dim/")

# Load Country Dimension
country_dim = spark.read.csv("wasbs://hockey@projecthockey.blob.core.windows.net/Dimensions/country_dim_csv/",
    header=True,
    inferSchema=True)

# Join source_file_1 with club_dim on Club Name
player_with_club = source_file_1.join(club_dim, source_file_1["Club Name"] == club_dim["Club_Name"], "left") \
    .select(
        source_file_1["Player Name"], 
        source_file_1["Position"], 
        source_file_1["Jersey_No"], 
        source_file_1["DOB"], 
        source_file_1["Nationality"], 
        club_dim["Club_ID"]
    ).distinct()

# Join the resulting DataFrame with country_dim on Country Name
player_with_club_country = player_with_club.join(country_dim, player_with_club["Nationality"] == country_dim["country_Name"], "left") \
    .select(
        player_with_club["Player Name"], 
        player_with_club["Position"], 
        player_with_club["Jersey_No"], 
        player_with_club["DOB"], 
        player_with_club["Club_ID"], 
        country_dim["Country_ID"]
    ).distinct()
player_with_club_country = player_with_club_country.dropDuplicates(["Player Name", "Club_ID", "Country_ID","Jersey_No","DOB","Jersey_No","Position"])
# Generate unique Player_ID with a scaling factor (e.g., starting from 100)
offset = 100
player_dim = player_with_club_country.withColumn("Player_ID", expr(f"monotonically_increasing_id() + {offset}"))

# Show the resulting player_dim
player_dim.show()

# Write the final player_dim to Azure Blob Storage
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/Dimensions/player_dim/"
player_dim.write.format("parquet").mode("overwrite").save(output_path)


+-----------------+--------+---------+---------+-------+----------+---------+
|      Player Name|Position|Jersey_No|      DOB|Club_ID|Country_ID|Player_ID|
+-----------------+--------+---------+---------+-------+----------+---------+
|            Bojan|      RW|       60|12-Feb-82|     16|         5|      100|
|       Bernd Leno|      GK|        2|24-Sep-80|     16|         5|      101|
|     Patrice Evra|      LB|       29|14-Mar-81|     13|         9|      102|
|   Tallulah Leach|      CB|       28|28-Mar-85|     14|         6|      103|
|         Valencia|      RW|       98|20-Dec-83|     21|         2|      104|
|          Marcelo|      LB|       85|20-Mar-82|     16|         4|      105|
|      Daley Blind|     CAM|       81|26-Jun-81|      7|         9|      106|
|       Dani Alves|      RB|       10|16-Nov-81|     10|        10|      107|
|         Tim Krul|      GK|       96|12-Oct-83|     11|         4|      108|
|SebastianGiovinco|       F|       45|23-Apr-83|     16|        

In [0]:
import pyspark.sql.functions as F

# 2. Read Time Dimension Files
time_file_1 = spark.read.csv("/mnt/hockey1/Hockey_Src1.csv", header=True, inferSchema=True)  # Month, Year
time_file_2 = spark.read.csv("/mnt/hockey1/Hockey_Src2.csv", header=True, inferSchema=True)  # Year
time_file_3 = spark.read.csv("/mnt/hockey1/DIM.Date.Table.csv", header=True, inferSchema=True)  # Quarter



In [0]:
# Step 1: Join Time File 1 (Month, Year) with Time File 2 (Year)
time_data_1 = time_file_1.join(time_file_2, on="Year", how="left").select("Year", "Month")




In [0]:
# Step 2: Rename column for consistency and join with Time File 3 (Quarter)
time_file_3 = time_file_3.withColumnRenamed("month_name", "Month")

# Step 3: Perform the final join with the combined DataFrame


In [0]:
time_data_combined = time_data_1.join(time_file_3, on="Month", how="left") \
    .select("Year", "Month", "quarter").distinct()



In [0]:
time_data_combined=time_data_combined.dropDuplicates(["Year", "Month", "quarter"])
# Step 4: Add a unique Time_ID by using monotonically increasing id and adding an offset
offset = 200  # Adjust as needed
time_dim= time_data_combined.withColumn("Time_ID", F.monotonically_increasing_id() + offset)



In [0]:
# Step 5: Show the Resulting DataFrame (containing Time_ID)
time_dim.show()



+----+---------+-------+-------+
|Year|    Month|quarter|Time_ID|
+----+---------+-------+-------+
|2012|     July|      3|    200|
|2012| November|      4|    201|
|2012| February|      1|    202|
|2012|  January|      1|    203|
|2012|    March|      1|    204|
|2012|  October|      4|    205|
|2012|      May|      2|    206|
|2012|   August|      3|    207|
|2012|    April|      2|    208|
|2012|     June|      2|    209|
|2012| December|      4|    210|
|2012|September|      3|    211|
|2011|     July|      3|    212|
|2011| November|      4|    213|
|2011| February|      1|    214|
|2011|  January|      1|    215|
|2011|    March|      1|    216|
|2011|  October|      4|    217|
|2011|      May|      2|    218|
|2011|   August|      3|    219|
+----+---------+-------+-------+
only showing top 20 rows



In [0]:
# Step 6: Save the Result to Azure Blob Storage (or another location)
output_path = "wasbs://hockey@projecthockey.blob.core.windows.net/Dimensions/time_dim/"


In [0]:
time_dim.write.format("parquet").mode("overwrite").save(output_path)

hive database

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS hockey_analysis")

DataFrame[]

In [0]:
league_dim.write.format("delta").mode("overwrite").saveAsTable("hockey_analysis.league_dim")

In [0]:
%sql
select * from hockey_analysis.league_dim;

League_Name League_ID NA 0 Bundesliga 1 EPL 2 LA Liga 3

In [0]:
club_dim.write.format("delta").mode("overwrite").saveAsTable("hockey_analysis.club_dim")

In [0]:
%sql
select * from hockey_analysis.club_dim;

Club_Name Manager Owner Club_ID League_ID Swanswa City Duncan Navarro Caleb Cote 0 0 Sunderland Zachary Burgess Marsden Clements 1 1 FC Koln Dante Castillo Cyrus Sutton 2 2 ManUtd Theodore Mcguire Baxter Stephens 3 3 Schalke Wanda Mayer Adam Strickland 4 4 Sevilla FC Tad Glenn Hamilton Cobb 5 5 Liverpool Kato Carver Wyatt Joyner 6 6 Valencia Phelan Rollins Fuller Odom 7 7 Hamburger SV Chelsea Mendoza Anthony Weeks 8 8 Real Sociedad Edward Chase Wyatt Hernandez 9 9 Real Madrid Tad Gardner Christopher Wilder 10 10 Chelsea Bruce Bray Beck Bell 11 11 Wreder Brewman Lacota Ramsey Brady Kelly 12 12 VFL Wolfsburg Upton Campbell Zephania Harding 13 13 Barcelona Malik Sanders Curran Drake 14 14 Dortmund Ursa Finch Yasir Conley 15 15 NA NA NA 16 16 Atheletico Bilbao Murphy Strong Erich Bird 17 17 Malaga FC Axel Farley Graiden Rowe 18 18 Levante Tyler Morin Tyler Carver 19 19 Aston Villa Hamish Clayton Sylvester Campos 20 20 Man City Damian Hancock Joshua Little 21 21 Bayern Munich Yvonne Bryan Erich Navarro 22 22 Everton Hayes Bowman Roth Rice 23 23 Atheletico Madrid Alvin Lucas Dean Green 24 24 NewCastle Finn Finch Emery Woods 25 25 Bayern Leverkusan Mollie Ferguson Hilel Nixon 26 26 RCD Espanyol Jason Hahn Maxwell Bates 27 27 VFB Stuttgart Kaseem Marsh Todd Emerson 28 28 Hannover 96 Chava Rosales Harlan Dudley 29 29 Arsenal Caesar Townsend Brandon Townsend 30 30

In [0]:
from pyspark.sql import functions as F

# Sample DataFrame
country_dim_csv = spark.read.csv("/mnt/hockey1/Dimensions/country_dim_csv", header=True, inferSchema=True)

# Show the original schema
country_dim_csv.printSchema()

# Rename columns with invalid characters (spaces, commas, etc.)
cleaned_columns = [F.col(c).alias(c.replace(" ", "_").replace(",", "_").replace(";", "_")) for c in country_dim_csv.columns]

# Apply the renamed columns
country_dim_csv_cleaned = country_dim_csv.select(*cleaned_columns)

# Show the cleaned schema
country_dim_csv_cleaned.printSchema()

# Verify that the renaming was applied
country_dim_csv_cleaned.show()


root
 |-- country_Name: string (nullable = true)
 |-- Coach: string (nullable = true)
 |-- Country_ID: integer (nullable = true)

root
 |-- country_Name: string (nullable = true)
 |-- Coach: string (nullable = true)
 |-- Country_ID: integer (nullable = true)

+------------+----------------+----------+
|country_Name|           Coach|Country_ID|
+------------+----------------+----------+
|      Brasil|   Cody Delacruz|         0|
|     England| Tobias Melendez|         1|
|     Belgium|    Lucas Hodges|         2|
|       Spain|Prescott Sweeney|         3|
|    Portugal|   William Price|         4|
|   Argentina|    Damon Pruitt|         5|
|     Germany|  Barry Reynolds|         6|
|          NA|              NA|         7|
|      Mexico|  Kuame Gallegos|         8|
|       Italy|    Elmo Patrick|         9|
| Netherlands|      Will Smith|        10|
+------------+----------------+----------+



In [0]:


# Save the DataFrame as a table
country_dim_csv.write.format("delta").mode("overwrite").saveAsTable("hockey_analysis.country_dim_csv")

In [0]:
%sql
select * from hockey_analysis.country_dim_csv;

country_Name Coach Country_ID Germany Barry Reynolds 0 NA NA 1 Argentina Damon Pruitt 2 Belgium Lucas Hodges 3 Italy Elmo Patrick 4 Spain Prescott Sweeney 5 Mexico Kuame Gallegos 6 Brasil Cody Delacruz 7 England Tobias Melendez 8 Portugal William Price 9 Netherlands Will Smith 10

In [0]:
match_dim.write.format("delta").mode("overwrite").saveAsTable("hockey_analysis.match_dim")

In [0]:
%sql
select * from hockey_analysis.match_dim;

Match_Name Match_ID International 0 Club 1

In [0]:
# Check encoding when reading the file
player_dim = spark.read.option("charset", "UTF-8").parquet("/mnt/hockey1/Dimensions/player_dim", header=True, inferSchema=True)


In [0]:
from pyspark.sql import functions as F

# Rename columns to remove any unwanted characters
cleaned_columns = [F.col(c).alias(c.replace("\n", "_").replace("\r", "_").replace(" ", "_").replace(";", "_")) for c in player_dim.columns]

# Apply the cleaned column names
player_dim= player_dim.select(*cleaned_columns)

# Verify the schema and data
player_dim.printSchema()
player_dim.show()


root
 |-- Player_Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey_No: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Club_ID: long (nullable = true)
 |-- Country_ID: integer (nullable = true)
 |-- Player_ID: long (nullable = true)

+-----------------+--------+---------+---------+-------+----------+---------+
|      Player_Name|Position|Jersey_No|      DOB|Club_ID|Country_ID|Player_ID|
+-----------------+--------+---------+---------+-------+----------+---------+
|            Bojan|      RW|       60|12-Feb-82|     16|         5|      100|
|       Bernd Leno|      GK|        2|24-Sep-80|     16|         5|      101|
|     Patrice Evra|      LB|       29|14-Mar-81|     13|         9|      102|
|   Tallulah Leach|      CB|       28|28-Mar-85|     14|         6|      103|
|         Valencia|      RW|       98|20-Dec-83|     21|         2|      104|
|          Marcelo|      LB|       85|20-Mar-82|     16|         4|      105|
|      Daley 

In [0]:
player_dim.write.format("delta").option("mergeSchema", "true").mode("overwrite").saveAsTable("hockey_analysis.player_dim")


In [0]:
player_dim.write.format("delta").mode("overwrite").saveAsTable("hockey_analysis.player_dim")

In [0]:
%sql
select * from hockey_analysis.player_dim;

Player_Name Position Jersey_No DOB Club_ID Country_ID Player_ID Bojan RW 60 12-Feb-82 16 5 100 Bernd Leno GK 2 24-Sep-80 16 5 101 Patrice Evra LB 29 14-Mar-81 13 9 102 Tallulah Leach CB 28 28-Mar-85 14 6 103 Valencia RW 98 20-Dec-83 21 2 104 Marcelo LB 85 20-Mar-82 16 4 105 Daley Blind CAM 81 26-Jun-81 7 9 106 Dani Alves RB 10 16-Nov-81 10 10 107 Tim Krul GK 96 12-Oct-83 11 4 108 SebastianGiovinco F 45 23-Apr-83 16 2 109 Joe Hart GK 46 16-Jun-83 14 4 110 Evans LB 56 6-Mar-82 24 0 111 Marco CDM 17 5-Feb-85 26 1 112 Joel Campbell RW 13 13-Jun-82 9 10 113 Francesco F 55 17-Oct-82 16 6 114 Raphael Varane CB 96 27-Oct-81 4 8 115 Mario Gomez ST 73 23-Jan-84 16 10 116 Filipe luis LB 76 28-Sep-80 24 0 117 Barthez GK 51 25-Nov-81 16 3 118 Edin Dzeko ST 86 13-Nov-80 19 0 119 Lionel Messi F 33 18-Feb-81 11 4 120 Cafu CB 38 7-Nov-80 28 6 121 Jerome Boateng CB 96 26-Dec-83 20 9 122 Isco CAM 43 19-Mar-83 27 4 123 Samir Handanovic GK 66 24-Jan-81 29 10 124 Joe Hart GK 46 16-Jun-83 27 4 125 Ashley Cole LB 57 21-Sep-82 16 5 126 Eden Hazard LW 75 16-Jun-83 16 6 127 Ramires CDM 92 25-Aug-85 24 8 128 Aguero F 51 14-May-84 7 9 129 Francesco F 55 17-Oct-82 15 6 130 Ulysses Blair RW 60 23-Jul-82 16 1 131 Rafael RB 68 3-Apr-84 9 10 132 Petr Cech GK 23 26-Sep-81 2 0 133 Robben RW 94 12-Jun-84 29 3 134 Filipe luis LB 76 28-Sep-80 16 0 135 Mats Hummels CB 22 25-Jun-82 24 8 136 Ronaldinho RW 60 1-Sep-82 23 0 137 Ezra Rojas CDM 41 27-Apr-84 28 6 138 Robben RW 94 12-Jun-84 22 3 139 Javier Manquillo RB 4 5-Jul-83 15 6 140 Lindegaard GK 11 9-Jun-82 16 5 141 Tallulah Leach CB 28 28-Mar-85 16 6 142 Paulo Dybala F 13 22-Jul-84 30 8 143 Iker Casillas GK 78 2-Oct-83 6 1 144 Fabio LB 96 31-Jan-83 12 1 145 SebastianGiovinco F 45 23-Apr-83 21 2 146 Willian LW 31 16-Dec-82 16 8 147 Emiliano LB 34 4-Jul-81 22 10 148 Viktor Fischer LW 71 13-Apr-83 6 2 149 David De Gea GK 34 27-Feb-85 7 5 150 Philipp Lahm RB 93 9-Sep-84 20 9 151 Manuel Neuer GK 28 7-Apr-82 6 2 152 Jordi Alba LB 81 14-Feb-83 2 0 153 Mario Mandzukic ST 3 21-Jun-84 6 1 154 Cafu CB 38 7-Nov-80 16 6 155 Gerrad Pique CB 35 11-Dec-80 3 10 156 Hugo Lloris GK 64 26-Feb-84 16 0 157 Gabriel F 82 16-Nov-83 16 6 158 Jose Enrique LB 34 22-Feb-83 16 6 159 Robben RW 94 12-Jun-84 16 3 160 Olivier Girioud ST 78 6-Mar-85 25 5 161 Ribery RW 66 7-Sep-84 16 3 162 Fletcher CAM 28 23-Jul-82 16 1 163 Thibaut Courtois GK 79 24-Jan-84 5 9 164 Giggs LW 11 6-Mar-84 2 0 165 Willian LW 31 16-Dec-82 2 8 166 Erik Lamela RW 38 25-Aug-84 25 5 167 Vladimir LW 8 16-Aug-85 17 2 168 Luke Shaw RB 57 26-Dec-82 1 10 169 David Alaba LB 24 7-Apr-81 16 5 170 Cristiano Ronaldo LW 55 8-Jul-81 16 8 171 Daniel Sturridge ST 27 17-May-85 23 0 172 Santiago Arias RB 84 13-Feb-82 16 10 173 Ferdinand LB 76 28-Sep-80 24 0 174 Phelan Sampson RB 65 18-Jul-82 28 4 175 Jamea F 19 3-Jan-82 22 3 176 Thiago Silva CB 66 5-Apr-84 2 0 177 Robert Lewandoski ST 98 21-Nov-81 16 1 178 Daniel Sturridge ST 27 17-May-85 16 0 179 Alessandro Florenzi RW 26 23-Jun-85 27 4 180 Phelan Sampson RB 65 18-Jul-82 11 4 181 Alexis Sanchez RW 85 24-Feb-83 7 9 182 Mario Gotze CAM 3 20-Nov-81 21 1 183 Jonas F 48 13-Dec-83 16 2 184 Fredericka Cleveland CAM 73 3-Aug-85 10 3 185 Wayne Rooney F 91 10-Oct-80 4 0 186 Manuel Neuer GK 28 7-Apr-82 12 2 187 Vidic CB 76 13-May-82 28 6 188 Alessio Cerci F 84 31-Jul-81 8 9 189 Ima Hoover RB 16 30-Apr-84 16 10 190 Juan Manuel CAM 82 10-Oct-80 9 3 191 Oscar CAM 63 26-Mar-83 11 4 192 Hulk RW 80 22-Apr-81 26 1 193 Luis Suarez ST 88 11-Feb-82 16 9 194 Isco CAM 43 19-Mar-83 14 4 195 Gareth Bale F 7 28-Jun-82 29 3 196 Antonio Cassano F 70 5-Jan-85 27 6 197 Etienne CDM 88 7-Sep-84 12 2 198 Hugo RB 52 12-Jun-84 10 3 199 Lionel Messi F 33 18-Feb-81 16 4 200 Mario Mandzukic ST 3 21-Jun-84 16 1 201 Thiago Silva CB 66 5-Apr-84 16 0 202 Buffon GK 44 20-Mar-85 24 8 203 Sandro CDM 74 2-Mar-81 16 0 204 Javier Mascherano CB 21 4-Jun-82 5 5 205 Jamea F 19 3-Jan-82 16 3 206 Kayler Navas GK 75 14-Apr-82 16 8 207 Robert Lewandoski ST 98 21-Nov-81 6 1 208 Hugo RB 52 12-Jun-8

In [0]:
time_dim.write.format("delta").mode("overwrite").saveAsTable("hockey_analysis.time_dim")

In [0]:
player_dim = spark.table("hockey_analysis.player_dim")
club_dim = spark.table("hockey_analysis.club_dim")
country_dim = spark.table("hockey_analysis.country_dim_csv")
match_dim = spark.table("hockey_analysis.match_dim")
time_dim = spark.table("hockey_analysis.time_dim")


In [0]:
league_dim.printSchema()
club_dim.printSchema()
country_dim.printSchema()
match_dim.printSchema()
time_dim.printSchema()
player_dim.printSchema()

root
 |-- League_Name: string (nullable = true)
 |-- League_ID: long (nullable = false)

root
 |-- Club_Name: string (nullable = true)
 |-- Manager: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Club_ID: long (nullable = true)
 |-- League_ID: long (nullable = true)

root
 |-- country_Name: string (nullable = true)
 |-- Coach: string (nullable = true)
 |-- Country_ID: integer (nullable = true)

root
 |-- Match_Name: string (nullable = true)
 |-- Match_ID: long (nullable = true)

root
 |-- Year: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- Time_ID: long (nullable = true)

root
 |-- Player_Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey_No: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Club_ID: long (nullable = true)
 |-- Country_ID: integer (nullable = true)
 |-- Player_ID: long (nullable = true)



In [0]:
from pyspark.sql.functions import monotonically_increasing_id
source_file_1 = spark.read.csv("/mnt/hockey1/Hockey_Src1.csv", header=True, inferSchema=True)

In [0]:
from pyspark.sql.functions import col

fact_player_statistics = (
    source_file_1.alias("sf1")
    .join(player_dim.alias("pd"), source_file_1["Player Name"] == player_dim["Player_Name"], "left")
    .join(club_dim.alias("cd"), source_file_1["Club Name"] == club_dim["Club_Name"], "left")
    .join(country_dim.alias("cnd"), source_file_1["Country Name"] == country_dim["Country_Name"], "left")
    .join(match_dim.alias("md"), source_file_1["Match"] == match_dim["Match_Name"], "left")
    .join(time_dim.alias("td"), source_file_1["Year"] == time_dim["Year"], "left")
    .select(
        monotonically_increasing_id().alias("Fact_ID"),
        col("td.Time_Id").alias("Time_Id"),
        col("md.Match_Id").alias("Match_Id"),
        col("pd.Player_Id").alias("Player_Id"),
        col("cd.Club_Id").alias("Club_Id"),
        col("cnd.Country_Id").alias("Country_Id"),
        col("sf1.Appearances").alias("Appearances"),
        col("sf1.Goals scored").alias("Goals_scored"),
        col("sf1.Goals assist").alias("Goals_assist"),
        col("sf1.Total shots").alias("Total_shots"),
        col("sf1.Shots on target").alias("Shots_on_target"),
        col("sf1.Fouls made").alias("Fouls_made"),
        col("sf1.Fouls suffered").alias("Fouls_suffered"),
        col("sf1.Yellow card").alias("Yellow_card"),
        col("sf1.Red card").alias("Red_card"),
        col("sf1.Goals saved").alias("Goals_saved"),
        col("sf1.Goals conceded(stopped)").alias("Goals_conceded"),
        col("sf1.Total penalty").alias("Total_penalty"),
        col("sf1.Successful penalty").alias("Successful_penalty"),
        col("sf1.Salary").alias("Salary")
    ).distinct()
)
# fact_player_statistics = fact_player_statistics.dropDuplicates(
#     ["Player_Id", "Club_Id", "Country_Id", "Match_Id"]
# )

fact_player_statistics.show()




+-------+-------+--------+---------+-------+----------+-----------+------------+------------+-----------+---------------+----------+--------------+-----------+--------+-----------+--------------+-------------+------------------+----------+
|Fact_ID|Time_Id|Match_Id|Player_Id|Club_Id|Country_Id|Appearances|Goals_scored|Goals_assist|Total_shots|Shots_on_target|Fouls_made|Fouls_suffered|Yellow_card|Red_card|Goals_saved|Goals_conceded|Total_penalty|Successful_penalty|    Salary|
+-------+-------+--------+---------+-------+----------+-----------+------------+------------+-----------+---------------+----------+--------------+-----------+--------+-----------+--------------+-------------+------------------+----------+
|     48|    223|       1|      541|      9|         7|          8|           7|           1|         75|             47|        10|             2|          1|       2|         17|            28|            4|                 2| $3,953.88|
|     71|    211|       1|      541|    

In [0]:
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/facttable/fact_player_statistics/"
fact_player_statistics.write.format("parquet").mode("overwrite").save(output_path)

In [0]:
fact_player_statistics.write.format("delta").mode("overwrite").saveAsTable("hockey_analysis.fact_player_statistics")

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
source_file_2 = spark.read.csv("/mnt/hockey1/Hockey_Src2.csv", header=True, inferSchema=True)

In [0]:
league_dim.printSchema()

root
 |-- League_Name: string (nullable = true)
 |-- League_ID: long (nullable = false)



In [0]:
from pyspark.sql.functions import col

fact_team_statistics = (
    source_file_2.alias("sf2")
    .join(league_dim.alias("ld"), source_file_2["League_Name"] == league_dim["League_Name"], "left")
    .join(club_dim.alias("cd"), source_file_2["Club_Name"] == club_dim["Club_Name"], "left")
    .join(country_dim.alias("cnd"), source_file_2["Country Name"] == country_dim["Country_Name"], "left")
    .join(match_dim.alias("md"), source_file_2["Match_Name"] == match_dim["Match_Name"], "left")
    .join(time_dim.alias("td"), source_file_2["Year"] == time_dim["Year"], "left")
    .select(
        monotonically_increasing_id().alias("Fact_ID"),
        col("td.Time_Id").alias("Time_Id"),
        col("md.Match_Id").alias("Match_Id"),
        col("ld.league_Id").alias("Player_Id"),
        col("cd.Club_Id").alias("Club_Id"),
        col("cnd.Country_Id").alias("Country_Id"),
        col("sf2.appearances").alias("appearances"),
        col("sf2.wins").alias("wins"),
        col("sf2.losts").alias("losts"),
        col("sf2.drawn").alias("drawn"),
        col("sf2.clean sheets").alias("clean_sheets"),
        col("sf2.Net Worth").alias("Net_Worth")
    ).distinct()
)

fact_team_statistics.show()

+-------+-------+--------+---------+-------+----------+-----------+----+-----+-----+------------+-----------+
|Fact_ID|Time_Id|Match_Id|Player_Id|Club_Id|Country_Id|appearances|wins|losts|drawn|clean_sheets|  Net_Worth|
+-------+-------+--------+---------+-------+----------+-----------+----+-----+-----+------------+-----------+
|    219|    235|       1|        1|     22|         7|          8|   8|    0|    0|           5|  $457,645 |
|    349|    211|       1|        2|     23|         7|          8|   2|    4|    2|           0|  $345,233 |
|    542|    235|       1|        3|     18|         7|          8|   1|    4|    3|           0|  $745,634 |
|    902|    211|       0|        0|     16|         2|          8|   7|    0|    1|           3|$7,657,455 |
|     23|    223|       1|        3|     24|         7|          8|   6|    2|    0|           2|  $563,466 |
|    463|    211|       1|        1|     29|         7|          8|   5|    3|    0|           3|  $856,847 |
|    723| 

In [0]:
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/facttable/fact_team_statistics/"


In [0]:
fact_team_statistics.write.format("parquet").mode("overwrite").save(output_path)

In [0]:
fact_team_statistics.write.format("delta").mode("overwrite").saveAsTable("hockey_analysis.fact_team_statistics")

In [0]:
%sql
select * from hockey_analysis.league_dim

League_Name League_ID NA 0 Bundesliga 1 EPL 2 LA Liga 3

In [0]:
%sql
select * from hockey_analysis.club_dim

Club_Name Manager Owner Club_ID League_ID Swanswa City Duncan Navarro Caleb Cote 0 0 Sunderland Zachary Burgess Marsden Clements 1 1 FC Koln Dante Castillo Cyrus Sutton 2 2 ManUtd Theodore Mcguire Baxter Stephens 3 3 Schalke Wanda Mayer Adam Strickland 4 4 Sevilla FC Tad Glenn Hamilton Cobb 5 5 Liverpool Kato Carver Wyatt Joyner 6 6 Valencia Phelan Rollins Fuller Odom 7 7 Hamburger SV Chelsea Mendoza Anthony Weeks 8 8 Real Sociedad Edward Chase Wyatt Hernandez 9 9 Real Madrid Tad Gardner Christopher Wilder 10 10 Chelsea Bruce Bray Beck Bell 11 11 Wreder Brewman Lacota Ramsey Brady Kelly 12 12 VFL Wolfsburg Upton Campbell Zephania Harding 13 13 Barcelona Malik Sanders Curran Drake 14 14 Dortmund Ursa Finch Yasir Conley 15 15 NA NA NA 16 16 Atheletico Bilbao Murphy Strong Erich Bird 17 17 Malaga FC Axel Farley Graiden Rowe 18 18 Levante Tyler Morin Tyler Carver 19 19 Aston Villa Hamish Clayton Sylvester Campos 20 20 Man City Damian Hancock Joshua Little 21 21 Bayern Munich Yvonne Bryan Erich Navarro 22 22 Everton Hayes Bowman Roth Rice 23 23 Atheletico Madrid Alvin Lucas Dean Green 24 24 NewCastle Finn Finch Emery Woods 25 25 Bayern Leverkusan Mollie Ferguson Hilel Nixon 26 26 RCD Espanyol Jason Hahn Maxwell Bates 27 27 VFB Stuttgart Kaseem Marsh Todd Emerson 28 28 Hannover 96 Chava Rosales Harlan Dudley 29 29 Arsenal Caesar Townsend Brandon Townsend 30 30

In [0]:
%sql
select * from hockey_analysis.match_dim

Match_Name Match_ID International 0 Club 1

In [0]:
%sql
select * from hockey_analysis.country_dim_csv

country_Name Coach Country_ID Germany Barry Reynolds 0 NA NA 1 Argentina Damon Pruitt 2 Belgium Lucas Hodges 3 Italy Elmo Patrick 4 Spain Prescott Sweeney 5 Mexico Kuame Gallegos 6 Brasil Cody Delacruz 7 England Tobias Melendez 8 Portugal William Price 9 Netherlands Will Smith 10

In [0]:
%sql
select * from hockey_analysis.player_dim

Player_Name Position Jersey_No DOB Club_ID Country_ID Player_ID Bojan RW 60 12-Feb-82 16 5 100 Bernd Leno GK 2 24-Sep-80 16 5 101 Patrice Evra LB 29 14-Mar-81 13 9 102 Tallulah Leach CB 28 28-Mar-85 14 6 103 Valencia RW 98 20-Dec-83 21 2 104 Marcelo LB 85 20-Mar-82 16 4 105 Daley Blind CAM 81 26-Jun-81 7 9 106 Dani Alves RB 10 16-Nov-81 10 10 107 Tim Krul GK 96 12-Oct-83 11 4 108 SebastianGiovinco F 45 23-Apr-83 16 2 109 Joe Hart GK 46 16-Jun-83 14 4 110 Evans LB 56 6-Mar-82 24 0 111 Marco CDM 17 5-Feb-85 26 1 112 Joel Campbell RW 13 13-Jun-82 9 10 113 Francesco F 55 17-Oct-82 16 6 114 Raphael Varane CB 96 27-Oct-81 4 8 115 Mario Gomez ST 73 23-Jan-84 16 10 116 Filipe luis LB 76 28-Sep-80 24 0 117 Barthez GK 51 25-Nov-81 16 3 118 Edin Dzeko ST 86 13-Nov-80 19 0 119 Lionel Messi F 33 18-Feb-81 11 4 120 Cafu CB 38 7-Nov-80 28 6 121 Jerome Boateng CB 96 26-Dec-83 20 9 122 Isco CAM 43 19-Mar-83 27 4 123 Samir Handanovic GK 66 24-Jan-81 29 10 124 Joe Hart GK 46 16-Jun-83 27 4 125 Ashley Cole LB 57 21-Sep-82 16 5 126 Eden Hazard LW 75 16-Jun-83 16 6 127 Ramires CDM 92 25-Aug-85 24 8 128 Aguero F 51 14-May-84 7 9 129 Francesco F 55 17-Oct-82 15 6 130 Ulysses Blair RW 60 23-Jul-82 16 1 131 Rafael RB 68 3-Apr-84 9 10 132 Petr Cech GK 23 26-Sep-81 2 0 133 Robben RW 94 12-Jun-84 29 3 134 Filipe luis LB 76 28-Sep-80 16 0 135 Mats Hummels CB 22 25-Jun-82 24 8 136 Ronaldinho RW 60 1-Sep-82 23 0 137 Ezra Rojas CDM 41 27-Apr-84 28 6 138 Robben RW 94 12-Jun-84 22 3 139 Javier Manquillo RB 4 5-Jul-83 15 6 140 Lindegaard GK 11 9-Jun-82 16 5 141 Tallulah Leach CB 28 28-Mar-85 16 6 142 Paulo Dybala F 13 22-Jul-84 30 8 143 Iker Casillas GK 78 2-Oct-83 6 1 144 Fabio LB 96 31-Jan-83 12 1 145 SebastianGiovinco F 45 23-Apr-83 21 2 146 Willian LW 31 16-Dec-82 16 8 147 Emiliano LB 34 4-Jul-81 22 10 148 Viktor Fischer LW 71 13-Apr-83 6 2 149 David De Gea GK 34 27-Feb-85 7 5 150 Philipp Lahm RB 93 9-Sep-84 20 9 151 Manuel Neuer GK 28 7-Apr-82 6 2 152 Jordi Alba LB 81 14-Feb-83 2 0 153 Mario Mandzukic ST 3 21-Jun-84 6 1 154 Cafu CB 38 7-Nov-80 16 6 155 Gerrad Pique CB 35 11-Dec-80 3 10 156 Hugo Lloris GK 64 26-Feb-84 16 0 157 Gabriel F 82 16-Nov-83 16 6 158 Jose Enrique LB 34 22-Feb-83 16 6 159 Robben RW 94 12-Jun-84 16 3 160 Olivier Girioud ST 78 6-Mar-85 25 5 161 Ribery RW 66 7-Sep-84 16 3 162 Fletcher CAM 28 23-Jul-82 16 1 163 Thibaut Courtois GK 79 24-Jan-84 5 9 164 Giggs LW 11 6-Mar-84 2 0 165 Willian LW 31 16-Dec-82 2 8 166 Erik Lamela RW 38 25-Aug-84 25 5 167 Vladimir LW 8 16-Aug-85 17 2 168 Luke Shaw RB 57 26-Dec-82 1 10 169 David Alaba LB 24 7-Apr-81 16 5 170 Cristiano Ronaldo LW 55 8-Jul-81 16 8 171 Daniel Sturridge ST 27 17-May-85 23 0 172 Santiago Arias RB 84 13-Feb-82 16 10 173 Ferdinand LB 76 28-Sep-80 24 0 174 Phelan Sampson RB 65 18-Jul-82 28 4 175 Jamea F 19 3-Jan-82 22 3 176 Thiago Silva CB 66 5-Apr-84 2 0 177 Robert Lewandoski ST 98 21-Nov-81 16 1 178 Daniel Sturridge ST 27 17-May-85 16 0 179 Alessandro Florenzi RW 26 23-Jun-85 27 4 180 Phelan Sampson RB 65 18-Jul-82 11 4 181 Alexis Sanchez RW 85 24-Feb-83 7 9 182 Mario Gotze CAM 3 20-Nov-81 21 1 183 Jonas F 48 13-Dec-83 16 2 184 Fredericka Cleveland CAM 73 3-Aug-85 10 3 185 Wayne Rooney F 91 10-Oct-80 4 0 186 Manuel Neuer GK 28 7-Apr-82 12 2 187 Vidic CB 76 13-May-82 28 6 188 Alessio Cerci F 84 31-Jul-81 8 9 189 Ima Hoover RB 16 30-Apr-84 16 10 190 Juan Manuel CAM 82 10-Oct-80 9 3 191 Oscar CAM 63 26-Mar-83 11 4 192 Hulk RW 80 22-Apr-81 26 1 193 Luis Suarez ST 88 11-Feb-82 16 9 194 Isco CAM 43 19-Mar-83 14 4 195 Gareth Bale F 7 28-Jun-82 29 3 196 Antonio Cassano F 70 5-Jan-85 27 6 197 Etienne CDM 88 7-Sep-84 12 2 198 Hugo RB 52 12-Jun-84 10 3 199 Lionel Messi F 33 18-Feb-81 16 4 200 Mario Mandzukic ST 3 21-Jun-84 16 1 201 Thiago Silva CB 66 5-Apr-84 16 0 202 Buffon GK 44 20-Mar-85 24 8 203 Sandro CDM 74 2-Mar-81 16 0 204 Javier Mascherano CB 21 4-Jun-82 5 5 205 Jamea F 19 3-Jan-82 16 3 206 Kayler Navas GK 75 14-Apr-82 16 8 207 Robert Lewandoski ST 98 21-Nov-81 6 1 208 Hugo RB 52 12-Jun-8

In [0]:
%sql
select * from hockey_analysis.time_dim

Year Month quarter Time_ID 2012 November 4 200 2011 January 1 201 2012 February 1 202 2010 March 1 203 2011 August 3 204 2012 January 1 205 2010 December 4 206 2010 September 3 207 2011 February 1 208 2011 September 3 209 2010 February 1 210 2011 June 2 211 2010 January 1 212 2010 May 2 213 2012 September 3 214 2012 July 3 215 2012 August 3 216 2010 November 4 217 2011 October 4 218 2010 April 2 219 2012 May 2 220 2012 March 1 221 2011 December 4 222 2011 May 2 223 2011 July 3 224 2012 December 4 225 2012 June 2 226 2011 November 4 227 2012 April 2 228 2011 March 1 229 2012 October 4 230 2010 October 4 231 2010 July 3 232 2010 June 2 233 2011 April 2 234 2010 August 3 235

In [0]:
%sql
select distinct * from hockey_analysis.fact_player_statistics

Fact_ID Time_Id Match_Id Player_Id Club_Id Country_Id Appearances Goals_scored Goals_assist Total_shots Shots_on_target Fouls_made Fouls_suffered Yellow_card Red_card Goals_saved Goals_conceded Total_penalty Successful_penalty Salary 360 235 0 328 16 4 8 9 5 52 47 6 6 1 1 28 14 2 2 $19,374.96 2664 235 0 267 16 7 8 7 5 69 44 12 0 3 1 27 27 0 0 $3,454.41 376852 235 1 241 1 1 8 8 1 53 36 5 7 2 2 21 25 4 4 $3,643 346444 230 1 437 28 1 8 9 7 57 49 4 10 1 2 22 23 4 5 $6,606.16 207344 235 0 276 16 5 8 7 2 56 36 12 4 2 2 17 25 0 0 $14,571.68 53076 235 1 459 13 1 8 5 2 74 47 0 8 0 0 30 27 2 2 $5,460 377428 235 1 131 17 1 8 0 7 80 45 4 9 0 1 12 15 4 4 $10,000 406944 235 0 292 16 8 8 7 4 66 35 7 3 2 0 15 27 0 0 $1,568.15 256100 235 1 184 21 1 8 4 7 51 42 8 5 4 2 24 25 4 3 $9,000 159308 235 1 556 26 1 8 1 2 57 45 7 5 0 2 12 19 6 6 $18,000 257780 230 1 136 19 1 8 13 8 71 43 1 8 2 0 18 12 9 9 $18,964.93 157940 235 1 548 14 1 8 6 3 74 33 3 4 2 1 21 16 1 1 $4,534 1692 235 1 443 7 1 8 8 6 56 48 5 3 3 2 13 30 5 5 $10,000 258800 235 0 133 16 7 8 9 1 50 37 12 3 0 2 10 15 2 2 $3,353.65 100464 235 0 269 16 9 8 7 2 76 38 4 1 0 0 20 16 0 0 $2,038.46 1620 235 1 299 30 1 8 1 4 53 46 3 9 1 0 19 22 2 2 $20,000 99960 235 0 106 16 4 8 8 6 60 44 9 10 4 2 19 13 3 1 $12,601.74 2016 235 0 534 16 2 8 7 6 50 49 4 8 4 0 30 30 0 0 $19,723.42 207620 230 1 300 6 1 8 5 4 76 45 3 2 1 0 28 19 6 1 $19,074.73 209000 235 0 526 16 7 8 7 1 73 31 7 2 2 2 13 19 0 0 $11,669.93 346384 235 0 437 16 0 8 9 2 74 31 10 8 2 0 15 30 3 2 $6,606.16 345724 230 1 368 5 1 8 10 2 78 36 7 10 3 1 19 10 6 6 $12,314.30 314716 230 1 453 22 1 8 7 4 50 39 10 9 2 0 10 24 5 3 $18,143.01 4032 235 0 535 16 7 8 9 7 64 50 9 3 0 2 19 25 4 2 $7,056.71 159188 230 1 532 25 1 8 12 5 69 33 1 7 4 2 10 21 9 8 $16,965.68 147816 235 0 217 16 8 8 7 6 55 36 7 4 1 2 22 26 0 0 $7,902.24 345364 230 1 361 27 1 8 8 1 51 40 7 2 4 2 30 30 8 4 $12,316.70 159320 234 1 556 6 1 8 12 5 55 42 9 0 1 2 17 23 9 5 $17,000.93 209432 235 0 438 16 2 8 7 5 55 47 22 0 1 2 13 25 2 0 $15,030.93 208280 235 0 422 16 4 8 7 7 66 45 12 7 0 0 22 20 0 0 $16,965.68 346156 230 1 259 18 1 8 6 2 54 41 8 7 1 2 17 30 5 2 $9,460.48 258068 230 1 263 27 1 8 8 5 65 40 3 0 1 0 22 14 9 4 $9,014.81 51204 235 1 312 5 1 8 5 1 79 31 8 0 1 1 23 28 5 5 $20,000 304596 235 1 169 1 1 8 8 2 50 40 3 6 0 0 22 10 3 3 $6,500 313936 235 0 474 16 0 8 7 5 72 32 9 2 2 0 11 13 0 0 $18,765.14 98484 235 1 224 23 1 8 7 8 68 37 9 8 1 0 26 29 2 2 $10,000 158048 235 0 307 16 5 8 7 6 50 38 9 4 4 1 13 21 0 0 $15,193.72 101748 230 1 281 14 1 8 4 5 69 50 5 10 2 1 29 28 1 0 $18,046.47 99516 230 1 121 28 1 8 8 8 50 50 6 10 1 2 19 30 6 4 $12,725.61 216 235 0 471 16 9 8 7 5 66 49 4 5 0 0 21 20 0 0 $5,072.71 158540 230 1 426 3 1 8 5 4 65 40 5 10 3 1 13 18 1 1 $7,807.69 147888 235 0 145 16 8 8 7 8 74 44 12 7 2 0 15 19 0 0 $6,172.97 3024 235 0 418 16 6 8 9 8 77 50 10 7 4 2 10 28 2 2 $4,242.54 158972 230 1 306 21 1 8 12 7 70 35 3 8 2 1 18 15 8 8 $16,405.37 98580 230 1 218 14 1 8 7 1 59 34 5 10 1 1 23 14 3 3 $5,072.71 52452 230 1 360 13 1 8 6 8 67 50 8 2 3 1 17 15 5 2 $2,615.60 257252 235 1 112 26 1 8 6 7 74 42 7 10 4 1 19 17 0 0 $10,000 346360 234 1 480 1 1 8 7 7 63 35 9 6 1 1 23 18 6 4 $17,500.63 101460 230 1 117 24 1 8 6 5 53 48 7 2 3 1 20 26 2 2 $4,970.00 52176 235 0 350 16 6 8 9 8 77 50 10 7 4 2 10 28 2 2 $4,242.54 259292 230 1 132 10 1 8 8 3 52 48 6 9 0 2 19 21 6 4 $5,347.79 50952 235 0 179 16 7 8 7 8 64 37 12 0 4 0 11 20 2 0 $3,482.37 2952 235 0 568 16 9 8 9 2 79 39 7 2 3 0 11 18 2 2 $10,568.49 100680 235 0 238 16 7 8 7 3 80 31 10 6 2 0 18 12 0 0 $11,200.79 345412 235 1 338 15 1 8 0 4 56 46 2 10 1 0 15 26 3 3 $19,999 314860 230 1 376 18 1 8 6 2 54 41 8 7 1 2 17 30 5 2 $9,460.48 304836 230 1 279 7 1 8 10 7 68 45 0 7 0 2 10 25 7 6 $10,741.86 377716 235 1 188 11 1 8 1 3 59 43 9 9 2 1 19 22 3 3 $8,000 258944 235 0 326 16 5 8 7 4 50 46 6 9 0 1 13 27 0 0 $3,355.04 346588 230 1 304 6 1 8 4 3 65 34 6 7 0 1 21 23 6 0 $4,230.40 258116 235 1 282 18 1 8 3 8 50 40 3 7 2 1 29 25 2 2 $2,345 313864 235 

In [0]:
%sql
select * from hockey_analysis.fact_team_statistics

Fact_ID Time_Id Match_Id Player_Id Club_Id Country_Id appearances wins losts drawn clean_sheets Net_Worth 42 230 0 0 5 4 8 1 1 6 0 $6,456,456 120 230 1 1 0 1 8 2 3 3 1 $647,463 186 230 1 1 6 1 8 2 3 3 0 $856,847 323 230 1 3 17 1 8 4 1 3 0 $568,787 446 230 1 2 28 1 8 1 3 4 0 $345,233 580 235 0 0 5 9 8 4 2 2 2 $5,474,574 642 235 1 2 2 1 8 2 5 1 0 $674,576 1076 234 0 0 5 10 8 4 2 2 1 $7,574,554 1097 234 1 3 1 1 8 6 2 0 2 $785,674 1137 234 1 1 4 1 8 0 5 3 0 $634,623 1141 234 1 1 6 1 8 3 2 3 1 $456,354 1234 234 1 1 13 1 8 3 3 2 0 $745,635 1598 228 1 2 2 1 8 6 1 1 4 $764,533 2028 233 0 0 5 10 8 2 4 2 0 $8,567,434 2127 233 1 2 8 1 8 0 6 2 0 $568,647 2212 233 1 2 14 1 8 5 0 3 2 $756,745 2936 226 0 0 5 5 8 7 0 1 2 $7,845,675 3235 226 1 1 20 1 8 3 5 0 0 $457,635 3259 226 1 3 21 1 8 2 3 3 0 $875,675 3657 232 1 2 14 1 8 5 1 2 3 $756,745 3659 232 1 2 14 1 8 2 5 1 0 $756,745 4200 227 1 1 22 1 8 7 0 1 5 $645,634 4865 231 0 0 5 6 8 6 0 2 3 $5,678,647 4936 231 1 3 1 1 8 3 2 3 1 $745,635 5263 231 1 3 29 1 8 5 3 0 3 $745,634 5407 224 1 1 0 1 8 0 6 2 0 $756,734 5414 224 1 3 1 1 8 7 1 0 6 $785,674 5438 224 1 2 2 1 8 1 4 3 0 $865,745 5500 224 1 3 9 1 8 5 3 0 2 $756,734 5631 224 1 1 20 1 8 8 0 0 3 $636,466 5679 224 1 3 24 1 8 0 2 6 0 $756,745 5887 221 1 1 0 1 8 6 1 1 0 $647,463 6427 219 1 1 6 1 8 3 3 2 1 $636,466 6515 219 1 1 13 1 8 4 1 3 0 $634,623 7400 220 1 1 7 1 8 1 3 4 0 $457,356 7774 217 0 0 5 8 8 4 1 3 2 $4,576,456 7876 217 1 1 7 1 8 3 2 3 0 $456,346 8623 222 1 3 29 1 8 7 0 1 5 $563,466 8647 216 0 0 5 0 8 3 3 2 1 $5,476,456 8651 216 0 0 5 0 8 5 0 3 4 $5,476,456 8841 216 1 1 7 1 8 7 1 0 3 $457,356 9126 213 0 0 5 0 8 2 6 8 1 $7,856,756 9475 213 1 1 20 1 8 2 5 1 0 $567,457 9626 218 0 0 5 3 8 4 4 0 3 $7,645,764 9792 218 1 1 7 1 8 6 1 1 4 $785,674 9826 218 1 3 9 1 8 1 3 3 0 $756,734 10764 212 1 2 8 1 8 2 4 2 1 $568,647 11070 211 0 0 5 3 8 3 5 0 2 $7,645,764 11346 211 1 3 16 1 8 2 3 3 0 $346,456 11380 211 1 1 19 1 8 5 1 2 3 $634,623 11508 211 1 1 30 1 8 5 3 0 3 $346,346 11947 214 1 2 26 1 8 0 6 2 0 $870,034 12317 210 1 3 17 1 8 4 3 1 1 $745,634 12326 210 1 3 18 1 8 7 0 1 5 $745,653 12445 210 1 2 28 1 8 4 3 1 2 $567,455 12498 209 0 0 5 2 8 0 1 7 0 $6,546,344 12697 209 1 3 9 1 8 7 0 1 4 $756,734 13014 205 0 0 5 5 8 6 2 0 3 $7,845,675 13026 205 0 0 5 6 8 4 2 2 2 $5,674,565 13050 205 0 0 5 8 8 4 2 2 3 $7,457,655 13102 205 1 3 1 1 8 7 0 1 1 $573,535 13166 205 1 2 8 1 8 4 1 3 2 $764,576 13552 207 0 0 5 10 8 5 1 2 2 $8,567,434 13684 207 1 1 11 1 8 7 1 0 5 $346,346 14178 208 1 3 12 1 8 4 2 2 3 $756,745 14523 202 1 1 0 1 8 7 1 0 1 $647,463 14747 202 1 1 19 1 8 0 3 5 0 $356,346 14770 202 1 1 22 1 8 6 1 1 3 $457,457 14864 202 1 3 29 1 8 1 3 4 0 $567,856 15297 206 1 3 25 1 8 0 7 1 0 $634,633 15924 200 0 0 5 8 8 5 3 0 3 $7,457,655 16174 200 1 3 18 1 8 1 4 3 0 $457,457 16608 203 1 2 14 1 8 4 2 2 1 $756,745 16794 203 1 1 30 1 8 6 0 2 3 $564,563 16846 201 0 0 5 4 8 4 1 3 1 $7,567,455 16869 201 0 0 5 6 8 1 1 6 1 $6,747,456 16918 201 0 0 5 10 8 6 2 0 4 $7,574,554 16990 201 1 1 6 1 8 0 6 2 0 $456,354 17047 201 1 1 11 1 8 7 1 0 5 $346,456 205 230 1 2 8 1 8 6 1 1 4 $764,576 507 235 0 0 5 3 8 4 3 1 3 $7,567,454 693 235 1 2 8 1 8 2 3 3 1 $568,647 694 235 1 2 8 1 8 1 5 2 0 $568,647 998 234 0 0 5 4 8 6 1 1 4 $7,567,455 1028 234 0 0 5 6 8 2 4 2 1 $6,747,456 1069 234 0 0 5 10 8 5 1 2 2 $7,574,554 1301 234 1 1 19 1 8 3 2 3 1 $634,623 1413 234 1 2 28 1 8 6 2 0 1 $745,634 1955 233 0 0 5 3 8 5 3 0 1 $7,567,454 2670 229 1 1 13 1 8 4 4 0 1 $745,635 2955 226 0 0 5 7 8 4 2 2 2 $7,456,454 3524 232 1 2 2 1 8 8 0 0 4 $674,576 4408 225 0 0 5 8 8 8 0 0 2 $7,457,655 4673 225 1 1 20 1 8 4 4 0 0 $457,635 5667 224 1 2 23 1 8 1 3 4 0 $745,634 5827 221 0 0 5 6 8 7 0 1 3 $5,674,565 6341 219 0 0 5 9 8 1 1 6 1 $5,474,574 6372 219 1 3 1 1 8 5 3 0 2 $745,635 6458 219 1 3 9 1 8 1 3 4 0 $456,354 6726 223 0 0 5 0 8 5 2 1 2 $5,674,577 6850 223 1 1 0 1 8 5 2 1 0 $756,734 6978 223 1 3 12 1 8 4 2 2 3 $756,745 7026 223 1 3 16 1 8 2 3 3 0 $346,456 7118 223 1 3 24 1 8 7 1 0 6 $756,745 7360 220 1 2 2 1 8 4 2 

In [0]:
dbutils.widgets.text("passw", "")

In [0]:
passw=dbutils.widgets.get("passw")

1.Player report with country and club they played for

In [0]:
report1=spark.sql("""
WITH player_country_club AS (
    SELECT DISTINCT  
        fps.Player_Id,
        pd.Player_Name AS player_na,
        fps.Country_Id AS country_ic,
        cnd.Country_Name AS country_n,
        fps.Club_Id AS club_id,
        cd.Club_Name AS club_name
    FROM hockey_analysis.fact_player_statistics fps
    inner JOIN hockey_analysis.player_dim pd ON fps.Player_Id = pd.Player_Id
    inner JOIN hockey_analysis.country_dim_csv cnd ON fps.Country_Id = cnd.Country_Id
    inner JOIN hockey_analysis.club_dim cd ON fps.Club_Id = cd.Club_Id
),
deduplicated_players AS (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY player_na, country_ic, country_n, club_id, club_name 
               ORDER BY Player_Id
           ) AS row_num
    FROM player_country_club
)
SELECT Player_Id, player_na, country_ic, country_n, club_id, club_name
FROM deduplicated_players
WHERE row_num = 1
ORDER BY player_na, Player_Id, country_ic, country_n;
""")

if len(passw)>=8:
  report1.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "report1",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

2.Player report who has maximum appearances in International matches

In [0]:
report2=spark.sql("""
WITH player_appearances AS (
    SELECT 
        p.Player_Id, 
        p.Player_Name, 
        SUM(f.Appearances) AS total_appearances
    FROM hockey_analysis.fact_player_statistics f
    JOIN hockey_analysis.player_dim p ON f.Player_Id = p.Player_Id
    left JOIN hockey_analysis.match_dim m ON f.Match_Id = m.Match_Id
    WHERE m.Match_name = 'International'
    GROUP BY p.Player_Id, p.Player_Name
)
SELECT Player_Id, Player_Name, total_appearances
FROM player_appearances
ORDER BY total_appearances DESC
LIMIT 1
""")
if len(passw)>=8:
  report2.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "report2",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

3.Find top 10 goal scorers of 2010

In [0]:

report3=spark.sql("""
WITH top_goal_scorers AS (
    SELECT 
        MIN(p.Player_Id) AS Player_Id,  -- Representative Player_Id
        p.Player_Name, 
        SUM(f.Goals_Scored) AS total_goals
    FROM hockey_analysis.fact_player_statistics f
    JOIN hockey_analysis.player_dim p ON f.Player_Id = p.Player_Id
    JOIN hockey_analysis.time_dim t ON f.Time_Id = t.Time_Id
    WHERE t.Year = 2010
    GROUP BY p.Player_Name
)
SELECT Player_Id, Player_Name, total_goals
FROM top_goal_scorers
ORDER BY total_goals DESC
LIMIT 10;
""")
if len(passw)>=8:
  report3.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "report3",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )


4.ind top 5 players who got red card

In [0]:
report4=spark.sql("""
WITH red_card_count AS (
    SELECT 
        MIN(p.Player_Id) AS Player_Id,  -- Representative Player_Id
        p.Player_Name, 
        SUM(f.Red_Card) AS total_red_cards
    FROM hockey_analysis.fact_player_statistics f
    JOIN hockey_analysis.player_dim p ON f.Player_Id = p.Player_Id
    GROUP BY p.Player_Name
)
SELECT Player_Id, Player_Name, total_red_cards
FROM red_card_count
ORDER BY total_red_cards DESC
LIMIT 5;
""")
if len(passw)>=8:
  report4.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "report4",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )



5.Find the top 5 successful clubs

In [0]:
report5=spark.sql("""
WITH club_success AS (
    SELECT 
        c.Club_Id, 
        c.Club_Name, 
        SUM(f.Wins) AS total_wins
    FROM hockey_analysis.fact_team_statistics f
    JOIN hockey_analysis.club_dim c ON f.Club_Id = c.Club_Id
    GROUP BY c.Club_Id, c.Club_Name
)
SELECT Club_Name, total_wins
FROM club_success
ORDER BY total_wins DESC
LIMIT 5
""")
if len(passw)>=8:
  report5.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "report5",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

In [0]:
if len(passw)>=8:
  league_dim.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "league_dim",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

In [0]:
if len(passw)>=8:
   club_dim.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "club_dim",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

In [0]:
if len(passw)>=8:
  country_dim.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "country_dim",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

In [0]:
if len(passw)>=8:
  match_dim.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "match_dim",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

In [0]:
if len(passw)>=8:
  player_dim.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "player_dim",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

In [0]:
if len(passw)>=8:
  time_dim.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "time_dim",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

In [0]:
if len(passw)>=8:
  fact_player_statistics.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "fact_player_statistics",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

In [0]:
if len(passw)>=8:
  fact_team_statistics.\
    write.jdbc(
      url = "jdbc:sqlserver://hockeyanalysis.database.windows.net;databaseName=hockeyanalysis",
      table = "fact_team_statistics",
      mode = "append",
      properties = {
                    "user" : 'hockeyserver',
                    "password" : passw}
      )

In [0]:
%sql
DROP TABLE IF EXISTS `dbo.league_dim`;


In [0]:
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/Output/report1/"

# Write DataFrame to the container
report1.write.format("csv").mode("overwrite").save(output_path)

In [0]:
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/Output/report2/"

# Write DataFrame to the container
report2.write.format("csv").mode("overwrite").save(output_path)

In [0]:
report2.show()

+---------+-----------+-----------------+
|Player_Id|Player_Name|total_appearances|
+---------+-----------+-----------------+
|      166|    Willian|             3456|
+---------+-----------+-----------------+



In [0]:
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/Output/report3/"

# Write DataFrame to the container
report3.write.format("csv").mode("overwrite").save(output_path)

In [0]:
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/Output/report4/"

# Write DataFrame to the container
report4.write.format("csv").mode("overwrite").save(output_path)

In [0]:
output_path = f"wasbs://hockey@projecthockey.blob.core.windows.net/Output/report5/"

# Write DataFrame to the container
report5.write.format("csv").mode("overwrite").save(output_path)